# MATH&ML-6 Математический анализ в контексте задачи оптимизации. Часть 3
###  Содержание <a class="anchor" id=0></a>

- [1. О чём лекция](#2)
- [2. Градиентный спуск: применения и классификации](#2)
- [3. Метод Ньютона](#3)
- [4. Квазиньютоновские методы](#4)
- [5. Линейное программирование](#5)
- [6. Практика: Линейное программирование](#6)
- [7. Практика: Дополнительные методы оптимизации](#7)
- [8. Итоги](#8)


# 1. О чём лекция <a class="anchor" id=1></a>

[к содержанию](#0)

* рассмотрим, какие вариации существуют у уже известного вам алгоритма градиентного спуска, и узнаем, в чём суть **обратного распространения ошибки**;

* познакомимся с **методом Ньютона** и **квазиньютоновскими методами BFGS** и **L-BFGS**;

* разберём область применения задач **линейного программирования** и попрактикуемся в их решении;

* узнаем, что такое **метод отжига** и **метод координатного спуска**.

>Из предыдущего модуля вы помните, что в каких-то методах оптимизации мы использовали лишь значение функции, где-то — считали градиент, а где-то — находили матрицы производных. Эти особенности дают возможность поделить все алгоритмы на **три группы**:
>
>* методы нулевого порядка (их работа основана на оценке значений самой целевой функции в разных точках);
>
>* методы первого порядка (при работе они используют первые производные в дополнение к информации о значении функции);
>
>* методы второго порядка (для них необходимо оценивать и значение функции, и значение градиента, и гессиан (матрицу Гессе)).

**Примечание**. Иногда в литературе можно встретить термины «оракул первого порядка» или «оракул нулевого порядка». Так обозначают компоненты алгоритма, которые находят информацию на каждом шаге для метода соответствующего порядка.

В данном модуле мы будем рассматривать методы разных порядков и практически в каждом юните будем практиковаться в их применении. В конце мы обобщим полученные знания и сравним, какие методы и в каких случаях показывают наилучшие результаты.

→ Важно отметить, что некоторые алгоритмы оптимизации мы пока не сможем использовать по прямому назначению: мы опробуем их на известных вам моделях линейной или логистической регрессии или на функциях одной или нескольких переменных, но основная сфера их применения — это искусственные нейронные сети. Тем не менее, понимание всего пула алгоритмов поможет вам в будущем без проблем подбирать необходимый метод вне зависимости от того, задачу какой сложности вы решаете: минимизируете простейшую функцию или обучаете нейронную сеть со сложной архитектурой.

***

**Задание 1.1**

Пусть прибыль в вашей компании выражается следующей функцией, которая зависит от параметра  — количества производимых товаров:

$f(x)=-x^{4}+6 x^{3}-4 x^{2}+80$

Найдите максимально возможную прибыль, которую вы можете получить, варьируя количество произведённых товаров.

In [29]:
from sympy import Symbol, Eq, diff, solve

x = Symbol('x')
fun = -x**4+6*x**3-4*x**2+80
df = diff(fun)
sol = solve(Eq(df,0))

print(f'x={sol[0]} => f={fun.subs(x,sol[0])} \nx={sol[1]} => f={fun.subs(x,sol[1])} \nx={sol[2]} => f={fun.subs(x,sol[2])}')


x=0 => f=80 
x=1/2 => f=1275/16 
x=4 => f=144


**Задание 1.4**

Допустим, вы хотите произвести некоторое количество товара, которое зависит от часов работы двух ключевых сотрудников следующим образом:

$f(x, y)=x^{2}+2 y^{2}$

Однако вы можете оплатить этим сотрудникам не более 20 часов работы.

Какое наибольшее количество товаров вы сможете произвести в таком случае?

In [46]:
from sympy import symbols, Eq, solve, diff

x, y, z = symbols('x y z') 
f = x**2+2*y**2 + z*(x+y-20) # запишем функцию Лагранжа
f_1 = x**2+2*y**2 # запишем исходную функцию

f_x = diff(f,x) # найдём частные производные от ф-и Ларанжа

f_y = diff(f,y)
f_z = diff(f,z)

sol = solve([Eq(f_x,0), Eq(f_y,0), Eq(f_z,0)], [x, y, z]) # разрешим систему уравнений собраную из частных производных
print(f'Первый работник: {round(sol[x])} ч/часов\nВоторой работник: {round(sol[y])} ч/часов')
print(f'Кол-во товаров: {f_1.subs([(x,round(sol[x])),(y,round(sol[y]))])} шт')

Первый работник: 13 ч/часов
Воторой работник: 7 ч/часов
Кол-во товаров: 267 шт


# 2. Градиентный спуск: применения и классификации <a class="anchor" id=2></a>

[к содержанию](#0)

# 3. Метод Ньютона <a class="anchor" id=3></a>

[к содержанию](#0)

# 4. Квазиньютоновские методы <a class="anchor" id=4></a>

[к содержанию](#0)

# 5. Линейное программирование <a class="anchor" id=5></a>

[к содержанию](#0)

# 6. Практика: Линейное программирование <a class="anchor" id=6></a>

[к содержанию](#0)

# 7. Практика: Дополнительные методы оптимизации <a class="anchor" id=7></a>

[к содержанию](#0)

# 8. Итоги <a class="anchor" id=8></a>

[к содержанию](#0)